In [0]:
import numpy as np
import csv
import torch
import torch.nn as nn
from torch.nn.utils.rnn import *
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils as utils
# import pickle as pk
from torch.utils.data import DataLoader, Dataset
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
nepochs = 20
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
% cd "/content/drive/My Drive/hw4p2"
!pip install -q kaggle
!mkdir -p ~/.kaggle
!touch ~/.kaggle
!cp kaggle.json ~/.kaggle/

!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

/content/drive/My Drive/hw4p2
kaggle.json


In [5]:
!mkdir "/content/data/"
% cd "/content/data/"
!kaggle competitions download -c 11785-fall19-hw4p2

/content/data
100% 2.18G/2.18G [00:41<00:00, 56.3MB/s]
100% 2.18G/2.18G [00:41<00:00, 57.0MB/s]
 79% 42.0M/52.9M [00:01<00:00, 22.2MB/s]
100% 52.9M/52.9M [00:01<00:00, 42.1MB/s]
  0% 0.00/3.48k [00:00<?, ?B/s]
100% 3.48k/3.48k [00:00<00:00, 2.91MB/s]
  0% 0.00/247k [00:00<?, ?B/s]
100% 247k/247k [00:00<00:00, 75.0MB/s]
100% 95.9M/95.9M [00:02<00:00, 44.3MB/s]

  0% 0.00/1.44M [00:00<?, ?B/s]
100% 1.44M/1.44M [00:00<00:00, 47.3MB/s]


In [6]:
!unzip '/content/data/dev_new.npy.zip'
!unzip '/content/data/test_new.npy.zip'
!unzip '/content/data/train_new.npy.zip'
!unzip '/content/data/train_transcripts.npy.zip'

Archive:  /content/data/dev_new.npy.zip
  inflating: dev_new.npy             
Archive:  /content/data/test_new.npy.zip
  inflating: test_new.npy            
Archive:  /content/data/train_new.npy.zip
  inflating: train_new.npy           
Archive:  /content/data/train_transcripts.npy.zip
  inflating: train_transcripts.npy   


In [7]:
path = "/content/data/"

speech_train = np.load(path + 'train_new.npy',
                       allow_pickle=True, encoding='bytes')
speech_valid = np.load(path + 'dev_new.npy',
                       allow_pickle=True, encoding='bytes')
speech_test = np.load(path + 'test_new.npy',
                      allow_pickle=True, encoding='bytes')

transcript_train = np.load(
    path + 'train_transcripts.npy', allow_pickle=True, encoding='bytes')
transcript_valid = np.load(
    path + 'dev_transcripts.npy', allow_pickle=True, encoding='bytes')
print("Data Loading Sucessful.....")

Data Loading Sucessful.....


In [0]:
letter_list = ['$', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
               'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',
               'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
               'X', 'Y', 'Z', '-', "'", '.', '_', '+',
               ' ', '<', '>']

In [0]:
def transform_letter_to_index(transcript, letter_list):
    '''
    :param transcript :(N, ) Transcripts are the text input
    :param letter_list: Letter list defined above
    :return letter_to_index_list: Returns a list for all
                                  the transcript sentence to index
    '''
    trans2idx = []
    for n, sentence in enumerate(transcript):
        sent2idx = []
        for token in sentence:
            tok2idx = []
            for char in token.decode('UTF-8'):
                tok2idx.append(letter_list.index(char))
            tok2idx.append(letter_list.index(' '))
            sent2idx.extend(tok2idx)
        # trans2idx.append([letter_list.index('<sos>')] + [letter_list.index(
        #     ' ')] + sent2idx[:] + [letter_list.index('<eos>')])
        trans2idx.append([letter_list.index('<')] +
                         sent2idx[:-1] + [letter_list.index('>')])
    return(trans2idx)

In [10]:
character_text_train = transform_letter_to_index(transcript_train, letter_list)
character_text_valid = transform_letter_to_index(transcript_valid, letter_list)
print("Transformed data sucessfully.....")

Transformed data sucessfully.....


In [0]:
class pBLSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim,
                             hidden_size=hidden_dim,
                             num_layers=1,
                             bidirectional=True)

    def forward(self, x, lens):
        '''
        :param x :(N,T, H1) input to the pBLSTM h
        :return output: (N,T,H) encoded sequence
                        from pyramidal Bi-LSTM
        '''
        x = x.transpose(0, 1)

        if x.shape[1] % 2 == 1:
            x = x[:, :-1, :]

        x = x.contiguous().view(-1,
                      x.shape[1] // 2,
                      x.shape[2] * 2)

        x = x.permute(1, 0, 2)
        x = pack_padded_sequence(x,
                                 lengths=lens // 2,
                                 batch_first=False,
                                 enforce_sorted=False)
        x, _ = self.blstm(x)
        return(x, _)


class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, value_size=128, key_size=128):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=1,
                            bidirectional=True)

        self.PyramidalBLSTM1 = pBLSTM(hidden_dim * 4, hidden_dim)
        self.PyramidalBLSTM2 = pBLSTM(hidden_dim * 4, hidden_dim)
        self.PyramidalBLSTM3 = pBLSTM(hidden_dim * 4, hidden_dim)

        self.key_network = nn.Linear(hidden_dim * 2, value_size)
        self.value_network = nn.Linear(hidden_dim * 2, key_size)

    def forward(self, utterance_padded, lens):
        rnn_inp = pack_padded_sequence(utterance_padded,
                                       lengths=lens,
                                       batch_first=False,
                                       enforce_sorted=False)
        outputs, _ = self.lstm(rnn_inp)
        outputs, lens = pad_packed_sequence(outputs)

        outputs, _ = self.PyramidalBLSTM1(outputs, lens)
        outputs, lens = pad_packed_sequence(outputs)

        outputs, _ = self.PyramidalBLSTM2(outputs, lens)
        outputs, lens = pad_packed_sequence(outputs)

        outputs, enc_finalstate = self.PyramidalBLSTM3(outputs, lens)
        encoded_inp, encoder_lens = pad_packed_sequence(outputs)

        # Use the outputs and pass it through the pBLSTM blocks
        keys = self.key_network(encoded_inp)
        value = self.value_network(encoded_inp)
        return (keys, value, encoder_lens, enc_finalstate)

In [0]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, query, key, value, lens):
        '''
        :param query :(N,context_size) Query is the output
                      of LSTMCell from Decoder
        :param key: (T,N,key_size) Key Projection from
                    Encoder per time step
        :param value: (T,N,value_size) Value Projection
                      from Encoder per time step
        :return output: Attended Context
        :return attention_mask: Attention mask that can be plotted
        '''

        # Compute (N, T) attention logits.
        # "bmm" stands for "batch matrix multiplication".
        # Input/output shape of bmm: (N, T, H), (N, H, 1) -> (N, T, 1)
        # ------------------energy------------------ #
        attention = torch.bmm(key.permute(1, 0, 2),
                              query.unsqueeze(2)).squeeze(2)

        # Create an (N, T) boolean mask for all padding positions
        # Make use of broadcasting: (1, T), (N, 1) -> (N, T)
        mask = torch.arange(key.size(0)).unsqueeze(
            0) >= lens.unsqueeze(1)
        mask = mask.to(device)
        attention.masked_fill_(mask, -1e9)

        # Take softmax over the "source length" dimension.
        # ------------------attention------------------ #
        attention = nn.functional.softmax(attention, dim=1)

        # Compute attention-weighted sum of context vectors
        # Input/output shape of bmm: (N, 1, T), (N, T, H) -> (N, 1, H)
        # ------------------context------------------ #
        out = torch.bmm(attention.unsqueeze(
            1), value.permute(1, 0, 2)).squeeze(1)
        return out, attention

In [0]:
def prune(state, score, beam, query, beamwidth=10):
    sortedscore = sorted(score, key=score.get, reverse=True)

    # Gets the lowest possible score to prune other branches
    threshold = score[sortedscore[beamwidth]]
    prunedstate = {}
    prunedscore = {}
    prunedquery = {}
    prunedbeam = []
    bestscore = -1e9
    bestpath = None

    for path in beam:
        if score[path] > threshold:
            prunedbeam += [path]  # set addition
            prunedstate[path] = state[path]
            prunedquery[path] = query[path]
            prunedscore[path] = score[path]
            if score[path] > bestscore:
                bestscore = score[path]
                bestpath = path
    return(prunedbeam, prunedscore, prunedstate, bestpath, prunedquery)

In [0]:
class Decoder(nn.Module):
    def __init__(self,
                 vocab_size,
                 hidden_dim,
                 value_size=128,
                 key_size=128,
                 isAttended=True,
                 ):
        super(Decoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_dim)

        self.lstm1 = nn.LSTMCell(input_size=hidden_dim + value_size,
                                 hidden_size=hidden_dim)

        self.lstm2 = nn.LSTMCell(input_size=hidden_dim,
                                 hidden_size=key_size)

        self.isAttended = isAttended
        if(isAttended):
            self.attention = Attention()

        self.character_prob = nn.Linear(key_size + value_size, vocab_size)

    def forward(self,
                key,
                values,
                enc_lens,
                text=None,
                train=True):
        '''
        :param key :(T,N,key_size) Output of the Encoder Key projection layer
        :param values: (T,N,value_size) Output of the
                        Encoder Value projection layer
        :param text: (N,text_len) Batch input of
                      text with text_length
        :param train: Train or eval mode
        :return predictions: Returns the character perdiction probability
        '''
        self.encoder_lens = enc_lens
        batch_size = key.shape[1]

        predictions = []
        # hidden_states = [None, None]
        prediction = torch.zeros(batch_size, 1)

        if(train):
            hidden_states = [None, None]
            max_len = text.shape[0]
            text = text.transpose(1, 0)

            for n, i in enumerate(range(max_len)):
                if i == 0:
                    query = torch.zeros((batch_size, key.shape[-1])).to(device)

                prob = np.random.random()
                if n > 0 and n < max_len - 1:
                    if prob < 0.8:
                        char_embed = self.embedding(text[:, i])
                    else:
                        char_embed = self.embedding(
                            prediction.argmax(dim=-1))
                else:
                    char_embed = self.embedding(text[:, i])

                context, attention = self.attention(query,
                                                    key,
                                                    values,
                                                    self.encoder_lens)

                inp = torch.cat([char_embed, context], dim=1)
                hidden_states[0] = self.lstm1(inp, hidden_states[0])
                inp_2 = hidden_states[0][0]
                hidden_states[1] = self.lstm2(inp_2, hidden_states[1])
                output = hidden_states[1][0]
                query = output
                prediction = self.character_prob(
                    torch.cat([output, context], dim=1))
                predictions.append(prediction.unsqueeze(1))
            concatentated_preds = torch.cat(predictions, dim=1)
            return concatentated_preds

        else:
            path = "<"
            beam = [path]
            pathscore = {"<": 0}
            state = {}
            query = {}

            # hidden_states = [None, None]
            # initial state (here 2 because of BLSTM)
            state[path] = [None, None]
            query[path] = torch.zeros((batch_size, key.shape[-1])).to(device)
            max_len = 10
            while True:  # Step forward
                nextbeam = []
                nextpathscore = {}
                nextstate = {}
                newquery = {}

                for path in beam:
                    # last character in the path
                    cfin = path[-1]

                    # hidden state generated by the LSTM
                    # when the previous character in
                    # this path was fed as input
                    hpath = state[path]

                    idx = torch.LongTensor([letter_list.index(cfin)]).to(device)
                    char_embed = self.embedding(idx)

                    # Calulates attention
                    context, attention = self.attention(query[path],
                                                        key,
                                                        values,
                                                        self.encoder_lens)

                    # The last symbol in every path being fed
                    # as input to the decoder LSTM.
                    inp = torch.cat([char_embed, context], dim=1)
                    h0 = self.lstm1(inp, hpath[0])
                    inp_2 = h0[0]
                    h1 = self.lstm2(inp_2, hpath[1])
                    output = h1[0]
                    # query = output

                    # needs to be passed through softmax
                    y = self.character_prob(
                        torch.cat([output, context], dim=1))
                    y = F.log_softmax(y, dim=1)
                    # h is the hidden state generated
                    # on passing the last char as input
                    h = [h0, h1]

                    # Loop through each character in the character set
                    for char in letter_list:
                        # This a string join
                        newpath = path + char
                        nextstate[newpath] = h
                        newquery[newpath] = output

                        # actually y[char_index]
                        nextpathscore[newpath] = pathscore[path] + \
                            y[0][letter_list.index(char)].item()

                        # Adds the new set of paths
                        nextbeam += [newpath]

                beam, pathscore, state, bestpath, query = prune(
                    nextstate, nextpathscore, nextbeam, newquery)
                if bestpath[-1] == ">" or len(bestpath)>=250:
                    return(bestpath)


In [0]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 input_dim,
                 vocab_size,
                 value_size=128,
                 key_size=128,
                 isAttended=True):
        super(Seq2Seq, self).__init__()

        self.encoder = Encoder(input_dim, 256)
        self.decoder = Decoder(vocab_size, 512)

    def forward(self,
                speech_input_padded,
                speech_len,
                text_padded,
                train=True):
        key, value, encoder_lens, enc_finalstate = self.encoder(speech_input_padded,
                                                                speech_len)
        if(train):
            predictions = self.decoder(key,
                                       value,
                                       encoder_lens,
                                       text=text_padded,
                                       train=True)
        else:
            predictions = self.decoder(key,
                                       value,
                                       encoder_lens,
                                       text=None,
                                       train=False)
        return(predictions)

In [0]:
# model = Seq2Seq(input_dim=40,
#                 vocab_size=len(letter_list)).to(device)
model = torch.load('/content/drive/My Drive/hw4p2/beam_check_venkat_14.pt').to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduction='none').to(device)

In [0]:
def train(model, train_loader, num_epochs, criterion, optimizer):
    model.train()
    print("Num epochs: ",num_epochs)
    for epochs in range(num_epochs):
        # loss_sum = 0
        start = time.time()

        for batch_num, (speech_padded, text_in_padded, text_label_padded, speech_len, text_len) in enumerate(train_loader):

            with torch.autograd.set_detect_anomaly(True):

                '''
                Please check if the text_input is of the
                (Batch_size, Sequence_length) i.e. (B,L)
                '''
                speech_padded = speech_padded.to(device)
                text_in_padded = text_in_padded.to(device)
                text_label_padded = text_label_padded.to(device)

                predictions = model(speech_padded, speech_len, text_in_padded)
                optimizer.zero_grad()

                mask = torch.zeros(
                    text_label_padded.transpose(1, 0).size()).to(device)  # BxL

                for i in range(len(text_len)):
                    mask[i, :text_len[i]] = 1
                mask = mask.view(-1).to(device)

                re = predictions.size(-1)
                predictions = predictions.contiguous().view(-1, re)

                # '''
                # If you do not have text_input as (B,L)
                # but have (L,B) instead then make
                # sure that you uncomment the next line of code
                # '''

                text_label_padded = text_label_padded.transpose(1, 0)  # BxL
                text_label_padded = text_label_padded.contiguous().view(-1)
                loss = criterion(predictions, text_label_padded)
                masked_loss = torch.sum(loss * mask)
                masked_loss.backward()

                # # utils.clip_grad_norm(model.parameters(), 2)
                optimizer.step()
                current_loss = float(masked_loss.item()) / \
                    int(torch.sum(mask).item())

                # print(current_loss)

                if batch_num % 25 == 1:
                    print('Batch num', batch_num,'train_loss', current_loss)
        print(f"Epoch {epochs+15} ran in {time.time()-start} seconds")
        torch.save(model, '/content/drive/My Drive/hw4p2/beam_check_venkat_' + str(epochs + 15) + '.pt')

In [0]:
# Dataloding related
class Speech2Text_Dataset(Dataset):
    def __init__(self, speech, text=None, train=True):
        self.speech = speech
        self.train = train
        if(text is not None):
            self.text = text

    def __len__(self):
        return self.speech.shape[0]

    def __getitem__(self, index):
        if(self.train):
            return (torch.tensor(self.speech[index].astype(np.float32)),
                    torch.tensor(self.text[index]))
        else:
            return torch.tensor(self.speech[index].astype(np.float32))


def collate_train(batch_data):

    speech = []
    speech_len = []
    text_in = []
    text_label = []
    text_len = []

    for s, t in batch_data:
        speech.append(s)
        speech_len.append(s.shape[0])
        text_in.append(t[:-1])
        text_label.append(t[1:])
        text_len.append(t.shape[0] - 1)

    speech_len = torch.LongTensor(speech_len)
    text_len = torch.LongTensor(text_len)

    speech_padded = pad_sequence(speech, batch_first=False)
    text_in_padded = pad_sequence(text_in, batch_first=False)
    text_label_padded = pad_sequence(text_label, batch_first=False)
    return(speech_padded, text_in_padded,
           text_label_padded, speech_len, text_len)

In [0]:
Speech2Text_train_Dataset = Speech2Text_Dataset(speech_train,
                                                character_text_train)
train_loader = DataLoader(Speech2Text_train_Dataset,
                          batch_size=128,
                          shuffle=True,
                          collate_fn=collate_train)

In [0]:
# train(model, train_loader, 16, criterion, optimizer) 

In [0]:
def test(model, test_loader):
  with torch.no_grad():
    model.eval()
    all_predictions = []

    for n, (speech_padded, speech_len) in enumerate(test_loader):
        predictions = model(speech_padded.to(device),
                            speech_len,
                            None,
                            train=False)
        # print(predictions)
        all_predictions.append(predictions[1:-1])
        print("Item", n)
    #     preds = predictions.argmax(dim=2)
    #     all_predictions.append(preds)
    # concatentated_preds = torch.cat(all_predictions, dim=0)
    # # print(concatentated_preds.shape)
    return(all_predictions)

In [0]:
def collate_test(batch_data):
    speech = []
    speech_len = []

    for s in batch_data:
        speech.append(s)
        speech_len.append(s.shape[0])

    speech_len = torch.LongTensor(speech_len)
    speech_padded = pad_sequence(speech,
                                 batch_first=False)
    return(speech_padded, speech_len)

In [0]:
Speech2Text_test_Dataset = Speech2Text_Dataset(speech_test, None, False)

test_loader = DataLoader(Speech2Text_test_Dataset,
                         batch_size=1,
                         shuffle=False,
                         collate_fn=collate_test)
# concatenated_preds = test(model, test_loader)
# print("Final shape is, ", concatenated_preds.shape)

In [23]:
concatenated_preds = test(model, test_loader)

Item 0
Item 1
Item 2
Item 3
Item 4
Item 5
Item 6
Item 7
Item 8
Item 9
Item 10
Item 11
Item 12
Item 13
Item 14
Item 15
Item 16
Item 17
Item 18
Item 19
Item 20
Item 21
Item 22
Item 23
Item 24
Item 25
Item 26
Item 27
Item 28
Item 29
Item 30
Item 31
Item 32
Item 33
Item 34
Item 35
Item 36
Item 37
Item 38
Item 39
Item 40
Item 41
Item 42
Item 43
Item 44
Item 45
Item 46
Item 47
Item 48
Item 49
Item 50
Item 51
Item 52
Item 53
Item 54
Item 55
Item 56
Item 57
Item 58
Item 59
Item 60
Item 61
Item 62
Item 63
Item 64
Item 65
Item 66
Item 67
Item 68
Item 69
Item 70
Item 71
Item 72
Item 73
Item 74
Item 75
Item 76
Item 77
Item 78
Item 79
Item 80
Item 81
Item 82
Item 83
Item 84
Item 85
Item 86
Item 87
Item 88
Item 89
Item 90
Item 91
Item 92
Item 93
Item 94
Item 95
Item 96
Item 97
Item 98
Item 99
Item 100
Item 101
Item 102
Item 103
Item 104
Item 105
Item 106
Item 107
Item 108
Item 109
Item 110
Item 111
Item 112
Item 113
Item 114
Item 115
Item 116
Item 117
Item 118
Item 119
Item 120
Item 121
Item 122
Ite

In [24]:
concatenated_preds[0]

"THE COMPANY ALSO DONATED FIFTY THOUSAND DOLLARS TO RELOCAL ENVIRONMENTAL GROUP TO HIGHER SPECIAL IS TO REVALUATE'S THE COMPANY'S PLANS TO CLEAD UP CONTAMINATED SITES"

In [0]:
idx = list(np.arange(523))
temp = [[a, b] for a, b in zip(idx, concatenated_preds)]
temp.insert(0, ['Id', 'Predicted'])

file = open('/content/drive/My Drive/hw4p2/sample_beam10_e14.csv',
            'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerows(temp)

In [0]:
letter_list = ['$', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
               'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',
               'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
               'X', 'Y', 'Z', '-', "'", '.', '_', '+',
               ' ', '<sos>', '<eos>']
print(letter_list.index(' '))

32


In [0]:
results = []
for sentence in concatenated_preds:
  txt =''
  for i in sentence:
    if i.item() == 34:
      break
    txt += letter_list[i.item()]
  results.append(txt)
print(results)

["THE COMPANY ALSO DONATED SIFTY THOUSAND DOLLARS TO A LOCAL ENVIRONMENTAL GROUP TO HIGHER SPECIALISTS TO EVALUATES THE COMPANY'S PLANS TO CLEAN UP CONTAMINATED SITES", "MR. BENTTIS CALL FOR MANDATORY TAPTING HASN'T BEEN ENDORSED BY THE WHITE HOUSE PERIOD", 'UNUSURELY SUSSER YESTERDAY SAID THAT THE LINE OF CREDIT WOULD BE GUERNINTEED BY PROVIVING SOLVER PERIOD', 'THE BUYOUT PLAN IS CONTINTIONS ON ALLEGANY RECEIVING THAT WAVER PERIOD', 'HOWEVER INCREASING THE COST OF RESEARCH ANALYSTS SHID MOT OF A. RESEARCHERS NOT TO WASTE THEM ON MERELY CHARIOUS OR REPETERED STUDIES', 'IN AUGUST COMMA A PACIFIC TELESIS GRAPUPUNITED MADE A SYMBALAR GARNETY TO THE COMMUNICATIONS WORKERS OF AMERICA COMMA PROVIDING TRAINING FOR NORPOSITIONS OF NECESSARY PERIOD', 'AN AMPICIATES IN NINETEEN HYPHEN NATION EFFORT TO HELP TWENTY YEARS OF WAR ANTEMPOTIOE ENDED WITHOUT AN AGREEMENT PERIOD', 'THE NEW CHAIRMAN NOW IS THAT IS PELITICAL CONNECTIONS MAKES SOME MARKET PARTICIPANTS SUSPICIOUS THAT YOU ALTRADISTIC MILLI